In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Getting the data using Pandas**

In [26]:
df=pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df.head()

In [27]:
df.describe()

**Counting positive and negative reviews**

In [28]:
df["sentiment"].value_counts()

In [29]:
df=df[:2000]

df["sentiment"].replace("positive",1,inplace=True)
df["sentiment"].replace("negative",0,inplace=True)


**Text Preprocessing**:
1- Remove html tags
2- Converting all review to lower case
3- Removing special character
4- Removing stop words
5- Stemming Words

**Remove tags**

In [30]:
import re
def remove_tags(text):
    clean=re.compile("<.*?>")
    re.sub(clean,"",text)
    return text
df["review"]=df["review"].apply(remove_tags)
df.head()

**Converting to lower**

In [31]:
def con_low(text):
    return text.lower()
df["review"]=df["review"].apply(con_low)
df.head()

**Removing special character**

In [32]:
def remove_special(text):
    x=""
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x+=" "
    return x
df["review"]=df["review"].apply(remove_special)
df.head()

**Removing stop words**

In [33]:
import nltk
from nltk.corpus import stopwords

def remove_stop(text):
    x=[]
    for i in text.split():
        if i not in stopwords.words("english"):
            x.append(i)
    return x
df["review"]=df["review"].apply(remove_stop)
df.head()

**Stemming Words**

In [34]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem(text):
    x=[]
    for i in text:
        x.append(ps.stem(i))
    return x
df["review"]=df["review"].apply(stem)
df.head()

****Joining Back****

In [35]:
def join_back(text):
    return " ".join(text)
df["review"]=df["review"].apply(join_back)
df.head()

**Selecting the features x and y**

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=500)
x=cv.fit_transform(df["review"]).toarray()
y=df.iloc[:,-1].values


**Splitting the dataset**

In [37]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.2)


**Applying naive bayes algorithm**

In [38]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
clf1=MultinomialNB()
clf2=BernoulliNB()
clf3=GaussianNB()
clf1.fit(xtrain,ytrain)
clf2.fit(xtrain,ytrain)
clf3.fit(xtrain,ytrain)

In [39]:
y1=clf1.predict(xtest)
y2=clf2.predict(xtest)
y3=clf3.predict(xtest)

**Checking the accuracy of our model**

In [40]:
from sklearn.metrics import accuracy_score
print("Multinomial",accuracy_score(ytest,y1))
print("Bernoulli",accuracy_score(ytest,y2))
print("Gaussian",accuracy_score(ytest,y3))

**Actual value vs Predicted value**

In [41]:
data=list(zip(ytest,y2))
ac_pred=pd.DataFrame(data,columns=["Actual","Predicted"])
print(ac_pred)